In [ ]:
import string

#Database extraction
import json
import psycopg2
from config import config
from sqlalchemy import create_engine
from config import config

#Data Analysis
import pandas as pd
pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 5000)
import matplotlib.pyplot as plt
import seaborn as sns

#Data Preprocessing and Feature Engineering
from textblob import TextBlob
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#Model Selection and Validation
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

In [ ]:
db_params = config()
conn = psycopg2.connect(**db_params)
db_cursor = conn.cursor()

raw_data = pd.read_sql_query('SELECT * FROM SanchezDictador', conn)

db_cursor.close()
conn.close()

In [ ]:
raw_data.head(5)

In [ ]:
df_raw = raw_data.copy()
print('cantidad de valores nulos: \n', df_raw.isnull().sum())

We don't have any null (NaN) at this point.

In [ ]:
df_processed = pd.read_csv('SanchezDictadorStopWords.csv')

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

# Remove useless punctuation
#for index, row in df_processed.iterrows():
    #df_processed['text_no_punct'] = df_processed["full_text"].apply(lambda text: remove_punctuation(text))
df_processed.head(5)

In [ ]:
STOPWORDS = set(stopwords.words('spanish'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

#for index, row in df_processed.iterrows():
    #df_processed['text_no_stop'] = df_processed["text_no_punct"].apply(lambda text: remove_punctuation(text))

In [ ]:
'''
df_sentiment = df_processed.copy()

for index, row in df_raw.iterrows():
    blob = TextBlob(row['full_text'])
    sentiment = blob.sentiment
    print(sentiment)
    df_sentiment['sentiment'] = sentiment.polarity
    df_sentiment['subjectivity'] = sentiment.subjectivity
'''